# Lab 3.1: LangGraph Persistence (Banking Assistant)

In this lab, we will add **Persistence** (Memory) to our graph. This allows the graph to remember the state (e.g., account context) across different interactions, enabling long-running conversations.

## Key Concepts
1. **Checkpointer**: A mechanism to save the state of the graph at every step.
2. **MemorySaver**: An in-memory checkpointer for development/testing.
3. **Thread ID**: A unique identifier to separate different user sessions.

In [4]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [9]:
# 3. Define Graph with Banking Persona
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage

class State(TypedDict):
    messages: Annotated[list, add_messages]
    
# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

# Banking Assistant Persona
sys_msg = SystemMessage(content="""You are a helpful Wells Fargo banking assistant. 
You can answer questions about account balances and transactions.
- If the user asks about a balance or transactions, ALWAYS check if they have specified the account type (Checking or Savings).
- If they haven't specified, ask them "Which account?".
- If they have specified (or it's clear from conversation history), provide a mock answer.
    - Checking Balance: $2,500
    - Savings Balance: $10,500
    - Checking Transactions: Walmart (-$50), Deposit (+$1000)
    - Savings Transactions: Interest (+$50)
""")

def chatbot(state: State):
    return {"messages": [llm.invoke([sys_msg] + state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

## 4. Add Persistence
We use `MemorySaver` to store the state.

In [10]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Compile with checkpointer
graph = graph_builder.compile(checkpointer=memory)

## 5. Run with Thread ID
We use `configurable` to specify the thread. In this initial interaction, we establish the context (Checking Account).

In [11]:
config = {"configurable": {"thread_id": "1"}}

# User establishes context
user_input = "I have a question about my Checking account."

for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Your **Checking account** balance is **$2,500**.  

**Recent transactions:**  
- Walmart: -$50  
- Deposit: +$1,000  

Let me know if you'd like more details or assistance! 😊


## 6. Verify Memory (Persistence)
Now we ask "What is the balance?" **without specifying the account**. The bot should remember we are talking about **Checking** because of the shared `thread_id`.

In [12]:
user_input = "What is the balance?"

# Note: We are using the same 'config' with thread_id='1'
for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Your **Checking account** balance is **$2,500**. Let me know if you'd like to review transactions or need further assistance! 😊


## 7. New Thread (No Memory)
If we change the `thread_id` to "2", the memory should be empty. The bot should NOT know which account we are referring to.

In [13]:
config_new = {"configurable": {"thread_id": "2"}}

# Asking the same question in a new thread
user_input = "What is the balance?"

for event in graph.stream({"messages": [("user", user_input)]}, config=config_new):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Which account?


In [14]:
user_input = "For the Savings account."

# Note: We are using the same 'config' with thread_id='1'
for event in graph.stream({"messages": [("user", user_input)]}, config=config_new):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: Your Savings account balance is **$10,500**.  

**Recent Savings transactions:**  
- Interest: +$50  

Let me know if you'd like further details! 😊
